In [2]:
import pandas as pd
import sys
print sys.version
import os
import glob

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jul  2 2016, 17:42:40) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [3]:
honda=pd.read_csv("../Data/OPP-115/annotations/1050_honda.com.csv", header=None)
honda.columns=["ID", "Type", "number", "number2", "number3", "category", "annotation", "date", "url"]
honda.head(2)

,ID,Type,number,number2,number3,category,annotation,date,url
0,20842,test_category_labeling_highlight_upitt,95,3836,0,Other,"{""Other Type"": {""selectedText"": ""STATEMENT OF ...",12/23/14,http://www.honda.com/site/site_privacy.aspx
1,15695,test_category_labeling_highlight_upitt,103,3836,1,Other,"{""Other Type"": {""selectedText"": ""American Hond...",12/23/14,http://www.honda.com/site/site_privacy.aspx


In [6]:
hondaAnnot=honda.loc[:,[ "category", "annotation"]]

In [7]:
group=list(hondaAnnot.groupby(["category"]))

In [43]:
annolist=group[3][1]["annotation"].values
print group[3][0]
eval(group[3][1]["annotation"].iloc[0]).keys()

First Party Collection/Use


['Collection Mode',
 'Identifiability',
 'Action First-Party',
 'Personal Information Type',
 'Choice Type',
 'Choice Scope',
 'Does/Does Not',
 'User Type',
 'Purpose']

In [14]:
for key in eval(annolist[0]):
  print key
  print eval(annolist[0])[key]

Security Measure
{'selectedText': 'American Honda has reasonable security measures in place to protect your personally identifiable information. This information is protected while being transmitted through a Secured Sockets Layer (SSL) connection. Most web pages that display personally identifiable information are encrypted using 128-bit SSL, which is an industry standard.', 'startIndexInSegment': 19, 'endIndexInSegment': 360, 'value': 'Secure data transfer'}


## Functions for packing annotations

In [13]:
## take two cateogries, put the values of the same key together in a list
def joinDict(origin, newdict):
  for key in newdict:
    origin.setdefault(key, [])
    origin[key]=origin[key]+[newdict[key]]
  return origin

## extract all the subcategory (topics) in a category, and gather all their attributes into lists
def packCategory(annotationList):
  annocollect={}
  for string in annotationList:
    anno=eval(string)
    for key in anno:
      annocollect.setdefault(key, {})
      annocollect[key]=joinDict(annocollect[key], anno[key])
  return annocollect

## pack each category in a list
def packPolicy(policyDataFrame):
  totalresult={}
  filtered=policyDataFrame.loc[:,[ "category", "annotation"]]
  grouped=list(filtered.groupby(["category"]))
  for group in grouped:
    category=group[0]
    annotationList=group[1]["annotation"].values
    totalresult[category]=packCategory(annotationList)
    
  return totalresult

def parseAllAnnotations(fileList):
  fulloutput={}
  for filepath in fileList:
    website=os.path.splitext(os.path.basename(filepath))[0].split("_")[1]
    annoData=pd.read_csv(filepath, header=None, names=["number", "Type", "ID", "number2", "number3", "category", "annotation", "date", "url"])
    parsedAnno=packPolicy(annoData)
    fulloutput[website]=parsedAnno
  return fulloutput


In [7]:
result=packPolicy(honda)
result["First Party Collection/Use"].keys()

Data Retention
Data Security
Do Not Track
First Party Collection/Use
International and Specific Audiences
Other
Policy Change
Third Party Sharing/Collection
User Access, Edit and Deletion
User Choice/Control


['Collection Mode',
 'Action First-Party',
 'Personal Information Type',
 'Does/Does Not',
 'User Type',
 'Purpose',
 'Choice Scope',
 'Choice Type',
 'Identifiability']

In [14]:
%%time
## pack all original annotations files
originalAnnoFiles=glob.glob("../Data/OPP-115/annotations/*.csv")
packedAnnoOriginal=parseAllAnnotations(originalAnnoFiles)

CPU times: user 3.26 s, sys: 16.6 ms, total: 3.28 s
Wall time: 3.28 s


In [22]:
print packedAnnoOriginal["foxsports.com"].keys()
print "\n"
print packedAnnoOriginal["foxsports.com"]["Data Retention"].keys()
print "\n"
print packedAnnoOriginal["foxsports.com"]["Data Retention"]["Retention Period"].keys()
print "\n"
print packedAnnoOriginal["foxsports.com"]["Data Retention"]["Retention Period"]

['Data Retention', 'First Party Collection/Use', 'User Choice/Control', 'Policy Change', 'Do Not Track', 'International and Specific Audiences', 'User Access, Edit and Deletion', 'Other', 'Data Security', 'Third Party Sharing/Collection']


['Personal Information Type', 'Retention Period', 'Retention Purpose']


['selectedText', 'startIndexInSegment', 'endIndexInSegment', 'value']


{'selectedText': ['We will retain your information for the period necessary to fulfill the purposes outlined in this Privacy Policy unless a longer retention period is required or allowed by applicable law.', 'We will retain your information for the period necessary to fulfill the purposes outlined in this Privacy Policy', 'Remember that even after you cancel your account, copies of some information from your account may remain viewable in some circumstances where, for example, you have shared information with Other Services. We may also retain backup information related to your account on our servers for s

In [24]:
import json
for key in packedAnnoOriginal:
  with open(os.path.join("../Data/parsed-annotation-original",key), "w") as f:
    json.dump(packedAnnoOriginal[key], f)

## Organize consolidated annotations

In [130]:
#/Users/NYao/Documents/MIDS/210/Data/OPP-115/consolidation/
hondaConso=pd.read_csv("../Data/OPP-115/consolidation/threshold-1.0-overlap-similarity/1050_honda.com.csv")
hondaConso.columns=["Number", "Type", "annotator", "number2", "number3", "category", "annotation", "date", "url"]
hondaConso.head(2)

,Number,Type,annotator,number2,number3,category,annotation,date,url
0,C549,test_category_labeling_highlight_fordham_aaaaa,121,3836,1,Other,"{""Other Type"": {""selectedText"": ""American Hond...",12/23/14,http://www.honda.com/site/site_privacy.aspx
1,15696,test_category_labeling_highlight_upitt,103,3836,2,Other,"{""Other Type"": {""selectedText"": ""This statemen...",12/23/14,http://www.honda.com/site/site_privacy.aspx


In [131]:
hondaConsoPack=packPolicy(hondaConso)

In [25]:
consoAnno10=glob.glob("../Data/OPP-115/consolidation/threshold-1.0-overlap-similarity/*.csv")
packedAnnoConso10=parseAllAnnotations(consoAnno10)

In [26]:
for key in packedAnnoConso10:
  with open(os.path.join("../Data/OPP-115/opp115-parsed-annotation-1.0",key), "w") as f:
    json.dump(packedAnnoConso10[key], f)

## Organize human-redable summaries